Подключаем и скачиваем библиотеки

In [ ]:
!pip install tensorflow
!pip install numpy
!pip install scipy
!pip3 install idx2numpy
from tensorflow import keras
from keras.models import Sequential
from keras import optimizers
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Reshape, LSTM, BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend as K
from keras.constraints import maxnorm
import numpy as np
from scipy.ndimage.measurements import center_of_mass
import math 
import cv2
import idx2numpy

Принимает целиком все изображение, находит в нем буквы и структуриет их по словам

In [64]:
from google.colab.patches import cv2_imshow as n

def vs(letters,height):
    last = 0
    a = []
    for i in range(len(letters) - 1):
        if letters[i+1][1] - letters[i][1] > height:
            a += [letters[last:i+1]]
            last = i + 1
        if i == len(letters) - 2:
            a += [letters[last:len(letters)]]
    return a


def vs_2(letters,weidth):
    last = 0
    a = []
    for i in range(len(letters)-1):
        if letters[i+1][0] - letters[i][0] > weidth*1.4:
            a += [letters[last:i+1]]
            last = i+1
        if i == len(letters)-2:
            a += [letters[last:len(letters)]]
    return a

def structure(letters):
    height = 0
    for i in letters:
        height += i[3]
    height = height/(2 *len(letters))

    letters  = vs(letters,height)

    for i in letters:
        i.sort(key=lambda x: x[0], reverse=False)

    for i in range(len(letters)):
        weight = 0
        for j in letters[i]:
            weight += j[2]
        weight = weight/len(letters[i])
        

        letters[i] = vs_2(letters[i],weight)


    return(letters)

def letters_extract(image_file: str, out_size=28):
    img = cv2.imread(image_file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 130, 255, cv2.THRESH_BINARY)
    img_erode = cv2.erode(thresh, np.ones((3, 3), np.uint8), iterations=1)

    #Получаем контуры
    contours, hierarchy = cv2.findContours(img_erode, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    output = img.copy()

    letters = []
    for idx, contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        if hierarchy[0][idx][3] == 0:
            cv2.rectangle(output, (x, y), (x + w, y + h), (70, 0, 0), 1)
            letter_crop = gray[y:y + h, x:x + w]
            # print(letter_crop.shape)

          
            size_max = max(w, h)
            letter_square = 255 * np.ones(shape=[size_max, size_max], dtype=np.uint8)
            if w > h:
                
                y_pos = size_max//2 - h//2
                letter_square[y_pos:y_pos + h, 0:w] = letter_crop
            elif w < h:
               
                x_pos = size_max//2 - w//2
                letter_square[0:h, x_pos:x_pos + w] = letter_crop
            else:
                letter_square = letter_crop

            # Меняем размер изображения как в датасете
            letters.append((x, y, w, h, cv2.resize(letter_square, (out_size, out_size), interpolation=cv2.INTER_AREA)))

    # сортируем по координате y
    letters.sort(key=lambda x: x[1], reverse=False)
    

    letters = structure(letters)

    return letters

letters = letters_extract('120.png')

emnist_labels = [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122]



считает веса сети (не запускать)

In [7]:


def emnist_model():
    model = Sequential()
    model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='valid', input_shape=(28, 28, 1), activation='relu'))
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(emnist_labels), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    return model

def emnist_train(model):
    emnist_path = '/content/drive/MyDrive/'
    X_train = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-train-images-idx3-ubyte')
    y_train = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-train-labels-idx1-ubyte')

    X_test = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-test-images-idx3-ubyte')
    y_test = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-test-labels-idx1-ubyte')

    X_train = np.reshape(X_train, (X_train.shape[0], 28, 28, 1))
    X_test = np.reshape(X_test, (X_test.shape[0], 28, 28, 1))

    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, len(emnist_labels))

    # тест
    k = 10
    X_train = X_train[:X_train.shape[0] // k]
    y_train = y_train[:y_train.shape[0] // k]
    X_test = X_test[:X_test.shape[0] // k]
    y_test = y_test[:y_test.shape[0] // k]

    X_train = X_train.astype(np.float32)
    X_train /= 255.0
    X_test = X_test.astype(np.float32)
    X_test /= 255.0

    x_train_cat = keras.utils.to_categorical(y_train, len(emnist_labels))
    y_test_cat = keras.utils.to_categorical(y_test, len(emnist_labels))

    
    learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

    model.fit(X_train, x_train_cat, validation_data=(X_test, y_test_cat), callbacks=[learning_rate_reduction], batch_size=64, epochs=30)



model = emnist_model()
emnist_train(model)
model.save('emnist_letters.h5')
    

(697932, 28, 28, 1) (697932,) (116323, 28, 28, 1) (116323,) 62
Epoch 1/30
1091/1091 [==============================] - 208s 190ms/step - loss: 4.0366 - accuracy: 0.0735 - val_loss: 3.9108 - val_accuracy: 0.1837 - lr: 0.0010
Epoch 2/30
1091/1091 [==============================] - 206s 188ms/step - loss: 3.7903 - accuracy: 0.1468 - val_loss: 3.5971 - val_accuracy: 0.2718 - lr: 0.0010
Epoch 3/30
1091/1091 [==============================] - 209s 192ms/step - loss: 3.4960 - accuracy: 0.2030 - val_loss: 3.2751 - val_accuracy: 0.3643 - lr: 0.0010
Epoch 4/30
1091/1091 [==============================] - 209s 192ms/step - loss: 3.2380 - accuracy: 0.2704 - val_loss: 2.9808 - val_accuracy: 0.4091 - lr: 0.0010
Epoch 5/30
1091/1091 [==============================] - 202s 185ms/step - loss: 2.9946 - accuracy: 0.3236 - val_loss: 2.7109 - val_accuracy: 0.4333 - lr: 0.0010
Epoch 6/30
1091/1091 [==============================] - 205s 188ms/step - loss: 2.7746 - accuracy: 0.3634 - val_loss: 2.4827 - val_a

Создаем и считаем готовую модель, после чего сохраняем ее

Функция которая используя предыдущие преобразование и нахождения букв, преобразует фотографию каждого символа в сам символ

In [65]:


def img_to_str(model, image_file: str):
    letters = letters_extract(image_file)
    s_out = ""
    for j in letters:
        for lett in j:    
            for i in range(len(lett)):
                s_out += emnist_predict_img(model, lett[i][4])
            s_out += ' '
        s_out += '\n'
    return s_out



Загружаем модель, после чего запускаем алгоритм распознавания

In [70]:
def emnist_predict_img(model, img):
    n(img)

    img_arr = np.expand_dims(img, axis=0)
    img_arr = 1 - img_arr/255.0
    img_arr[0] = np.rot90(img_arr[0], 3)
    img_arr[0] = np.fliplr(img_arr[0])
    img_arr = img_arr.reshape((1, 28, 28, 1))
    


    predict = model.predict([img_arr])
    result = np.argmax(predict, axis=1)
    return chr(emnist_labels[result[0]])


model = keras.models.load_model('emnist_letters.h5')
s_out = img_to_str(model, "120.png")
print('\n\nresult: ',s_out)

1/1 [==============================] - 0s 77ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 29ms/step


result:  nB110 n051d 

